# Exercise 3
# Semantic Table Interpretation

The following Jupyter Notebook contains coding tasks for Exercise 3. Fill up the code cells and execute to show output.
For this exercise, Pandas Library is required. For library reference, check for [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/index.html#user-guide). Also, check on [Pandas Cheatsheet](https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf).



In [1]:
# Install Pandas
# !pip install pandas

# Required Libraries
import pandas as pd
import requests
from SPARQLWrapper import SPARQLWrapper, JSON

# Task 3
## Column Data Type Detection

Task 3.1 : Given column are set of values, write a function to predict data types in the column and return result. For reference, look up on Lecture 3, pg-45

<img src="images/algorithm.png">


In [2]:
col = ["6", "1.8", "1.25"]
data_types = [int, float, str]

# Code Here
def predictDataType(records):
    types = {}
    for data_type in data_types:
        types[data_type] = 0
    for value in records:
        for data_type in data_types:
            try:
                data_type(value)
                types[data_type] = types[data_type] + 1
            except ValueError:
                pass
    for data_type in data_types:
        if types[data_type] == len(records):
            return data_type
    return None

res = predictDataType(col)
print(res)

<class 'float'>


# Task 5
### Automatic Semantic Table Interpretation

For this task requires API requests using 'requests' library. The table is provided as 'table2.csv'. For this task, the table is a multi-subject relational table
> - the first column is the subject column
> - the first two columns contain entities
> - the third column contains numbers

Using the file, perform the following tasks. 


## Task 5.1
Using MTab API provided, perform entity search on each cell and take out the top-ranked entity from results.
Refer to [MTab GitHub Documentation](https://github.com/phucty/mtab_tool/blob/master/docs/mtabes.md). 

In [4]:

api_endpoint = "https://mtab.kgraph.jp/api/v1/search?"
wd_endpoint = "http://www.wikidata.org/entity/"

def entitySearch(term):
    params = {
        "q": term,
        "m":"a",
        "limit": "3",
        "info": "1"
    }
    resp = requests.get(api_endpoint, verify=False, params=params)
    if resp.status_code == 200:
        r_data = resp.json()
        if r_data['status'] == "Success" and int(r_data['total']) > 0:
            hits = pd.DataFrame(r_data['hits'])
            hits['score'] = hits['score'].astype(float)
            max_score_idx = hits['score'].idxmax()
            dp_url = hits.iloc[max_score_idx].dp
            wd_id = hits.iloc[max_score_idx].id
            return (dp_url, wd_id)
    return 

# Code Here
data = pd.read_csv("data/table2.csv", header=None)
entity_list = {}

for term in data[0]:
    entity_list[term] = entitySearch(term)
    
for term in data[1]:
    entity_list[term] = entitySearch(term)
    
print(entity_list)

/Users/amirrezaalasti/.pyenv/versions/3.9.13/lib/python3.9/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'mtab.kgraph.jp'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/amirrezaalasti/.pyenv/versions/3.9.13/lib/python3.9/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'mtab.kgraph.jp'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/amirrezaalasti/.pyenv/versions/3.9.13/lib/python3.9/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'mtab.kgraph.jp'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#t

{'Berlin': None, 'Paris': None, 'Milano': None, 'Germany': None, 'France': None, 'Italy': None}


## Task 5.2 
For each pair of subject column, run the following query by replacing tags (#ENT1, #ENT2) with respective Wikidata URLs. Then run the queries with SPARQLwrapper and decide the property based on majority voting.

In [5]:
# For majority voting
def majorityVoting(list_of_values):
    """Counts the number of times each value appears in a list.
  
    Args:
      list_of_values: A list of values.
  
    Returns:
      A dictionary mapping each value to the number of times it appears in the list.
    """

    value_counts = {}
    for value in list_of_values:
        if value in value_counts:
            value_counts[value] += 1
        else:
            value_counts[value] = 1

    return value_counts

In [23]:
# Example template
wd_endpoint = "https://query.wikidata.org/sparql"

# Code Here
sparql_endpoint = SPARQLWrapper(wd_endpoint)
sparql_endpoint.setReturnFormat(JSON)
ent2ent= []

for ent1, ent2 in zip(data[0], data[1]):
    query1 = """
        SELECT ?property WHERE {
          wd:#ENT1 ?property wd:#ENT2 .
        }
    """
    query1 = query1.replace("#ENT1", entity_list[ent1][1])
    query1 = query1.replace("#ENT2", entity_list[ent2][1])
    sparql_endpoint.setQuery(query1)
    prop = sparql_endpoint.query().convert()
    prop = prop["results"]["bindings"][0]['property']["value"]
    ent2ent.append(prop)
    print(f"{ent1} -> {prop} -> {ent2}")
  
print(majorityVoting(ent2ent))  
    

Berlin -> http://www.wikidata.org/prop/direct/P17 -> Germany
Paris -> http://www.wikidata.org/prop/direct/P17 -> France
Milano -> http://www.wikidata.org/prop/direct/P17 -> Italy
{'http://www.wikidata.org/prop/direct/P17': 3}


## Task 5.3
For each pair of subject columns and any other entity column, Replace the tags (#ENT1, #NUM) with respective values and run the query in SPARQLWrapper. Decide the property based on majority voting.

In [19]:
# Code Here
ent2num = []

for idx, ent_key in enumerate(data[0]):
    num = str(data[2][idx])
    query2 = """
        SELECT ?property WHERE {
           wd:#ENT1 ?property "#NUM"^^xsd:decimal 
        }
    """
    query2 = query2.replace("#ENT1", entity_list[ent_key][1])
    query2 = query2.replace("#NUM", num)
    sparql_endpoint.setQuery(query2)
    prop2 = sparql_endpoint.query().convert()
    prop2 = prop2["results"]["bindings"]
    if len(prop2) > 0:
        prop2 = prop2[0]["property"]["value"]
        ent2num.append(prop2)
    print(f"{ent_key} -{prop2}-> {num}")
    
print(majorityVoting(ent2num))

Berlin -http://www.wikidata.org/prop/direct/P1082-> 3755251
Paris -[]-> 2140000
Milano -[]-> 1366180
{'http://www.wikidata.org/prop/direct/P1082': 1}


## Task 5.4 
Consider the following query template. Run the query for each entity and for each entity types, perform majority voting to detect type.

In [21]:
# Code Here
col1 = {}

for key in data[0]:
    entity = entity_list[key]
    query3 = """
        SELECT ?type WHERE {
          wd:#ENT1 wdt:P31 ?type .
        }
    """.replace("#ENT1", entity[1])
    sparql_endpoint.setQuery(query3)
    results = sparql_endpoint.query().convert()["results"]["bindings"]
    results = [uri['type']['value'] for uri in results]
    col1[key] = majorityVoting(results)

col2 = {}

for key in data[1]:
    entity = entity_list[key]
    query3 = """
        SELECT ?type WHERE {
          wd:#ENT1 wdt:P31 ?type .
        }
    """.replace("#ENT1", entity[1])
    sparql_endpoint.setQuery(query3)
    results = sparql_endpoint.query().convert()["results"]["bindings"]
    results = [uri['type']['value'] for uri in results]
    col2[key] = majorityVoting(results)
    
col1 = pd.DataFrame(col1).dropna()
print(col1)
print("")
print("")
col2 = pd.DataFrame(col2).dropna()
print(col2)

                                        Berlin  Paris  Milano
http://www.wikidata.org/entity/Q200250     1.0    1.0     1.0


                                         Germany  France  Italy
http://www.wikidata.org/entity/Q6256         1.0       1    1.0
http://www.wikidata.org/entity/Q3624078      1.0       1    1.0
